In [1]:
import os
#os.chdir('C:/Users/Mike/Desktop/word_embeddings-master') # windows dir
os.chdir('/home/mike/Desktop/Word Embeddings') # linux dir
import pandas as pd
from gensim.models import Word2Vec
import random

os.chdir('./Text Processing')
from TextPrep import TextPrep
from get_cosine import cosine_sim

os.chdir('../Meta Data')
from key_words import key_words, key_synonyms
from stop_words import stop_words
os.chdir('..')

In [4]:
# define functions for analyzing the word vectors
# returns the cosine similarity of two words
def cosine_sim(parser, keyword, text, labels, relative = False):
    # define tagged keywords. To generalize get a list of unique labels. loop through create a new variable for each label
    keyword_r = keyword + '_r'
    keyword_d = keyword + '_d'
    
    ptweets = []
    for i in range(len(text)):
        try:
            ptweets.append(parser.tag_keywords(keyword, text[i], labels[i])) # tweets and labels are global variables. change to local
        except Exception as e:
            print(e)
            print('failed at '+ keyword + str(i))
            
    # lemmatize
    ptweets = parser.multi_lemmatizer(ptweets, threads = 6)

    # drop single letters
    for i in range(len(ptweets)):
        ptweets[i] = [word for word in ptweets[i] if len(word) > 1]

    # train and save word2vec
    pmodel = Word2Vec(ptweets, window = 10, sg = 1)
    
    # return cosine similarity between the words
    if relative == True:
        return pmodel.wv.relative_cosine_similarity(keyword_r, keyword_d, topn=10)
    else:
        return pmodel.wv.similarity(keyword_r, keyword_d)

In [2]:
# load in data
meta_data = pd.read_csv('Meta Data/meta_data.csv')
tweet_df = pd.read_csv('Data/aggregated_tweets.csv')
# subset to tweets after oct 29
tweet_df = tweet_df[tweet_df['created'] >= '2019-11-06']
# merge data with meta data
tweet_df = pd.merge(tweet_df, meta_data, how = 'inner', on = 'user_id')
tweet_df = tweet_df[tweet_df.party.isin(['R', 'D'])].reset_index(drop=True)

tweets = tweet_df['text']
plabels = tweet_df['party']

In [3]:
# initialize text parser and pre-process tweets
prep = TextPrep(stopwords = stop_words, key_words = key_words, key_synonyms = key_synonyms)
tweets = [prep.twitter_preprocess(tweet) for tweet in tweets]

In [4]:
%%time
# run the permutation test
sim = []
for i in range(1):
    # permute the labels
    rlabels = random.sample(list(plabels), len(plabels))
    # Get cosine similarity
    rcosine = cosine_sim(parser = prep, keyword = 'trump', text = tweets, labels = rlabels, relative = True, threads = 2)
    # append to list
    sim.append(rcosine)

CPU times: user 1min 49s, sys: 856 ms, total: 1min 50s
Wall time: 1min 24s


In [19]:
# save results as a csv
#simdf = pd.DataFrame(sim, columns = ['cosine similarity'])
#simdf.to_csv('Analysis/permutation.csv', index = False)